In [1]:
#reset all variables when running again to avoid any mistakes

%reset -f

In [2]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import compute_class_weight

import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image

import tensorflow as tf

from keras.src.legacy.preprocessing.image import ImageDataGenerator

from matplotlib.ticker import StrMethodFormatter

In [3]:
# choose main hyperparameters here

#data / feature selections
balanced_flag = False

#Traing data splits :
test_split = 0.2
val_split = 0.20 # remember - this is fractional  after the test data has been split from the initial balanced sub dataset

# image parameters
target_size = (299,299) #pixel size to load img
#efficientnet V2S recommended img size 384 , https://www.kaggle.com/models/google/efficientnet-v2
#Inception V3 recommended image size 299x299

#select data augmentation
aug_flag = True
#augmentation params
horizontal_flip=False
vertical_flip=False
rotation_range=15
shear_range= 1
zoom_range = 0.07

#training
max_epochs = 100
loss_stop_patience = 7
learningRate = 0.001

loss_stop_patience_multi = [5,6,8] #patience in each step

#class weights
Use_class_weights = True

#early stopping
StoppingSelector = 'val_loss' #valid values: 'val_loss','val_f1' - add more here

#flags for models to include

Drift_fine_tune_flag = True
Drift_simple_flag = False

#set batch size according to balanc selection
if balanced_flag:
    batch_size = 8 #later player around with batch size to see how it affects performance
else:
    batch_size = 32 #hopefully speeds up training

#options: 'normal' :   tf.keras.losses.CategoricalCrossentropy()    , 'focal' : tf.keras.losses.CategoricalCrossentropy()
lossSelect = 'normal'

In [4]:
#make customizable string to add to dir for testing
if lossSelect == 'focal':
    custom_save_str = '_focalLoss'
else:
    custom_save_str = ''


In [5]:
# Path Definitions to relevant data + data loading

base_file_path = 'C:/Users/nikoLocal/Documents/Opencampus/Machine_Vision_challenge_data/'
image_path = base_file_path + '/input_train/input_train'

label_csv_name = 'Y_train_eVW9jym.csv'

#Loading .csv data to dataframes
train_df = pd.read_csv(os.path.join(base_file_path, label_csv_name))

In [6]:
# load well performing model
# adjust this path for the model - full path including *.keras
Trained_Model = tf.keras.models.load_model(os.path.join(base_file_path, 'model_evaluation\ImgSz_299_Aug_unbalanced_Cweights_focalLoss\InceptionV3_MultiPhase_full_fine_tune\model.keras'))

In [7]:
#DataFrame Preprocessing

#add another column to the dataframe according to dictionaries to map Labels correctly to numbers
dict_numbers = {'GOOD': 0,'Boucle plate':1,'Lift-off blanc':2,'Lift-off noir':3,'Missing':4,'Short circuit MOS':5}
dict_strings = {'GOOD': '0_GOOD','Boucle plate':'1_Flat loop','Lift-off blanc':'2_White lift-off','Lift-off noir':'3_Black lift-off','Missing':'4_Missing','Short circuit MOS':'5_Short circuit MOS'}

# for Test Data ("random submission" dataframe)
dict_strings_sub = {0: '0_GOOD',1:'1_Flat loop',2:'2_White lift-off',3:'3_Black lift-off',4:'4_Missing',5:'5_Short circuit MOS',6:'6_Drift'}

#list of all labels in the data
label_list = ['0_GOOD','1_Flat loop','2_White lift-off','3_Black lift-off','4_Missing','5_Short circuit MOS']

#create new columns in DFs via .map() method
train_df['LabelNum'] = train_df['Label'].map(dict_numbers)
train_df['LabelStr'] = train_df['Label'].map(dict_strings)

#number of classes
num_classes = len(label_list)

# get counts of label with the least entries
countList = train_df['LabelStr'].value_counts()
minCounts = countList.min()

BalancedDF = pd.DataFrame()
#concat sampled dataframes for each included label
for i in range(num_classes):
    BalancedDF = pd.concat([BalancedDF,train_df[train_df['LabelStr'] == label_list[i]].sample(n=minCounts)],axis=0)

#split dataframe according to fractional test size
train_df_balanced, test_df_balanced = train_test_split(BalancedDF, test_size=test_split, random_state=42) #keep random state constant to ensure
train_df_train, train_df_test = train_test_split(train_df, test_size=test_split, random_state=42) #keep random state constant to ensure

In [8]:
#Path definition and Dataframe preprocessing Test dataset (including Drift class)

# adjust this path to your machine / file structure
base_file_path = 'C:/Users/nikoLocal/Documents/Opencampus/Machine_Vision_challenge_data/'

# Import random Test Dataset
labelled_csv_path = 'manual_label_df_submission.csv' #this is needed only for the file name list
#labelled_csv_path = 'manually_labelled_data/johannes_0_500/manual_label_df_submission.csv' #this is needed only for the file name list

submission_image_path = base_file_path + '/input_test_1a4aqAg/input_test'

labelled_df = pd.read_csv(os.path.join(base_file_path, labelled_csv_path))

#remove potential nans
labelled_df = labelled_df.dropna(axis=0)

#extract only items with drift label class
Drift_df = labelled_df.loc[labelled_df['Manual_Label'] == 6]

#make a df that only has classes [0_Normal,1_Drift]
#Process label submission test dataframe
dict_strings_drift = {0: '0_Normal', 1: '0_Normal', 2: '0_Normal', 3: '0_Normal', 4: '0_Normal',
                    5: '0_Normal', 6: '1_Drift'}

labelled_df['Drift_Label'] = labelled_df['Manual_Label'].map(dict_strings_drift)

labelled_df['Drift_Label'].value_counts()

label_list_drift = ['0_Normal','1_Drift']

dict_Drift_Number = {'0_Normal':0, '1_Drift':1}

labelled_df['Drift_Label_num'] = labelled_df['Drift_Label'].map(dict_Drift_Number)

In [9]:
labelled_df['Drift_Label_num'].value_counts()

Drift_Label_num
0    998
1     55
Name: count, dtype: int64

In [10]:
#make a balanced DF Drift
#number of classes
num_classes_drift = 2 #don't need to count can hardcode

# get counts of label with the least entries
countList_drift = labelled_df['Drift_Label'].value_counts()
minCounts_drift = countList_drift.min()

BalancedDF_drift = pd.DataFrame()
#concat sampled dataframes for each included label
for i in range(num_classes_drift):
    BalancedDF_drift = pd.concat([BalancedDF_drift,labelled_df[labelled_df['Drift_Label'] == label_list_drift[i]].sample(n=minCounts_drift)],axis=0)

#make train/test split

#split dataframe according to fractional test size
train_df_balanced_drift, test_df_balanced_drift = train_test_split(BalancedDF_drift, test_size=test_split, random_state=42) #keep random state constant to ensure
train_df_drift, test_df_drift = train_test_split(labelled_df, test_size=test_split, random_state=42) #keep random state constant to ensure

In [11]:
#compute class weights for use of unbalanced datasets

class_numbers = np.unique(train_df_train['LabelNum'])

class_weights_unb = compute_class_weight(class_weight='balanced' ,classes = class_numbers,y=train_df_train['LabelNum'])
class_weights_b = compute_class_weight(class_weight='balanced' ,classes = class_numbers,y=train_df_balanced['LabelNum'])
equal_weights = np.ones(num_classes)

#make dicts that can be used by keras
class_w_unb_dict = dict(zip(class_numbers, class_weights_unb))
class_w_b_dict = dict(zip(class_numbers, class_weights_b))
class_w_equal_dict = dict(zip(class_numbers, equal_weights))

In [12]:
#compute class weights for use of unbalanced datasets. Test Dataset with Drift class

class_numbers_drift = np.unique(train_df_drift['Drift_Label_num'])

class_weights_unb_drift = compute_class_weight(class_weight='balanced' ,classes = class_numbers_drift,y=train_df_drift['Drift_Label_num'])
class_weights_b_drift = compute_class_weight(class_weight='balanced' ,classes = class_numbers_drift,y=train_df_balanced_drift['Drift_Label_num'])
equal_weights_drift = np.ones(2)

#make dicts that can be used by keras
class_w_unb_dict_drift = dict(zip(class_numbers_drift, class_weights_unb_drift))
class_w_b_dict_drift = dict(zip(class_numbers_drift, class_weights_b_drift))
class_w_equal_dict_drift = dict(zip(class_numbers_drift, equal_weights_drift))

In [13]:
# make a unique string (name) to save model and evaluation to file
# incorporate most important hyperparameters
# make a subfolder for one set of hyperparameters for more tidy folder and file structure

if aug_flag:
    augmentation_str = 'Aug'
else:
    augmentation_str = 'NoAug'

if balanced_flag:
    balance_str = 'balanced'
else:
    balance_str = 'unbalanced'

#class weights
if Use_class_weights:
    Cweights_str = '_Cweights'
else:
    Cweights_str = ''

hyperparam_name = 'ImgSz_{}_{}_{}{}{}'.format(target_size[0],augmentation_str,balance_str,Cweights_str,custom_save_str)
hyperparam_dir = os.path.join(base_file_path,'model_evaluation')
hyperparam_dir = os.path.join(hyperparam_dir,hyperparam_name)
#check if folder exists - if not create it
if not os.path.isdir(hyperparam_dir):
    os.makedirs(hyperparam_dir)

In [14]:
# initialize ImageDataGenerators
# use ImageDataGen because it has method flow_from_dataframe() that works really well together with pandas dataframes
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
# although deprecated the functionality can be used as discussed in feedback session

# HYPERPARAMTERS ########


class_mode = 'categorical' # how to store labels - either categorical (one-hot encoding) or as numbers
#class_mode = 'input'
labelCol = 'LabelStr'
#########################

#normalize pixel intensities
rescale = 1.0/255.0

datagen = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    rotation_range=0.0,
    shear_range=0.0,
    rescale=rescale,
    validation_split=val_split)

datagen_augmentation = ImageDataGenerator(
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip,
    rotation_range=rotation_range,
    shear_range= shear_range,
    zoom_range = zoom_range,
    rescale=rescale,
    validation_split=val_split)

datagen_test = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    rotation_range=0.0,
    shear_range=0.0,
    rescale=rescale,
    validation_split=0.0)


##########################################################

#unbalanced datasets

train_generator_unbalanced = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_val = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_unbalanced_aug = datagen_augmentation.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_val_aug = datagen_augmentation.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_unbalanced = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_unbalanced_metrics = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="grayscale",
    shuffle=False,
    seed=42,
    subset='training')

train_generator = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_val = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_aug = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_aug_val = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_metrics = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="grayscale",
    shuffle=False,
    seed=42,
    subset='training')

# generators for transfer learning - color mode is color here. Pretrained models expect color input

train_generator_unbalanced_color = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_val_color = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_unbalanced_color = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_aug_color = datagen_augmentation.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_unbalanced_metrics_color = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')

train_generator_color = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_val_color = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_aug_color = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_aug_val_color = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_color = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_metrics_color = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')

Found 5298 validated image filenames belonging to 6 classes.
Found 1324 validated image filenames belonging to 6 classes.
Found 5298 validated image filenames belonging to 6 classes.
Found 1324 validated image filenames belonging to 6 classes.
Found 1656 validated image filenames belonging to 6 classes.
Found 1656 validated image filenames belonging to 6 classes.
Found 272 validated image filenames belonging to 6 classes.
Found 68 validated image filenames belonging to 6 classes.
Found 272 validated image filenames belonging to 6 classes.
Found 68 validated image filenames belonging to 6 classes.
Found 86 validated image filenames belonging to 6 classes.
Found 86 validated image filenames belonging to 6 classes.
Found 5298 validated image filenames belonging to 6 classes.
Found 1324 validated image filenames belonging to 6 classes.
Found 1656 validated image filenames belonging to 6 classes.
Found 5298 validated image filenames belonging to 6 classes.
Found 1656 validated image filenam

In [15]:
#generators for test data (incl. drift class)

train_generator_drift = datagen.flow_from_dataframe(
    train_df_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='training')

train_generator_drift_aug = datagen_augmentation.flow_from_dataframe(
    train_df_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='training')

train_generator_drift_val = datagen.flow_from_dataframe(
    train_df_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='validation')

train_generator_drift_bal = datagen.flow_from_dataframe(
    train_df_balanced_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='training')

train_generator_drift_aug_bal = datagen_augmentation.flow_from_dataframe(
    train_df_balanced_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='training')

train_generator_drift_val_bal = datagen.flow_from_dataframe(
    train_df_balanced_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='validation')

test_generator_drift = datagen_test.flow_from_dataframe(
    test_df_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='training')

test_generator_drift_bal = datagen_test.flow_from_dataframe(
    test_df_balanced_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    keep_aspect_ratio = True,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb", #also needs to be identical to model used
    shuffle=False,
    seed=42,
    subset='training')

test_generator_drift_metrics = datagen_test.flow_from_dataframe(
    test_df_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    class_mode=class_mode,
    keep_aspect_ratio = True,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')

test_generator_drift_metrics_bal = datagen_test.flow_from_dataframe(
    test_df_balanced_drift,
    submission_image_path,
    x_col='filename',
    y_col='Drift_Label',
    target_size=target_size,
    class_mode=class_mode,
    keep_aspect_ratio = True,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')

Found 674 validated image filenames belonging to 2 classes.
Found 674 validated image filenames belonging to 2 classes.
Found 168 validated image filenames belonging to 2 classes.
Found 71 validated image filenames belonging to 2 classes.
Found 71 validated image filenames belonging to 2 classes.
Found 17 validated image filenames belonging to 2 classes.
Found 211 validated image filenames belonging to 2 classes.
Found 22 validated image filenames belonging to 2 classes.
Found 211 validated image filenames belonging to 2 classes.
Found 22 validated image filenames belonging to 2 classes.


In [16]:
# select training data based on flags

#for transfer learning
if balanced_flag:
    drift_model_val_gen = train_generator_drift_val_bal
    if aug_flag:
        drift_model_gen = train_generator_drift_aug_bal
    else:
        drift_model_gen = train_generator_drift_bal
else:
    drift_model_val_gen = train_generator_drift_val
    if aug_flag:
        drift_model_gen = train_generator_drift_aug
    else:
        drift_model_gen = train_generator_drift


if Use_class_weights:
    if balanced_flag:
        class_weights_drift = class_w_b_dict_drift
    else:
        class_weights_drift = class_w_unb_dict_drift
else:
    class_weights_drift = class_w_equal_dict_drift

In [17]:
# create a new model to bet trained to decide between "normal" and drift-class
# use the current best performing model as a base - both for model architecture as well as use the current weights

#Trained_Model.get_weights()

Drift_model = tf.keras.models.clone_model(Trained_Model)

Drift_model.set_weights(Trained_Model.get_weights())

Drift_model.pop() #removes last layer
Drift_model.add(tf.keras.layers.Dense(2, activation = 'softmax')) #add classification layer with 2 neurons for 2 classes

In [18]:
#compile the model
#select based on Str
if lossSelect == 'normal':
    loss_fun_drift = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_drift = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

#set only last layer to be trainable
for layer in Drift_model.layers[:]:
    layer.trainable = False

Drift_model.layers[-1].trainable = True

Drift_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), #start with "normal" learning rate
    loss=loss_fun_drift,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

Drift_model.compile()

Drift_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,806,882 (83.19 MB)

 Trainable params: 13,628,162 (51.99 MB)

 Non-trainable params: 8,178,720 (31.20 MB)

In [19]:
#train the drift model

if Drift_fine_tune_flag:

    if 'history_drift' in locals():
        del history_drift
    
    match StoppingSelector:
        case 'val_loss':
            StopCallback_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= 5,
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= 5,
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )
    
    history_drift = Drift_model.fit(
        drift_model_gen,
        validation_data = drift_model_val_gen,
        epochs=10,
        class_weight = class_weights_drift,
        callbacks=[StopCallback_drift],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/10
674/674 - 118s - 175ms/step - accuracy: 0.6647 - f1_score: 0.6370 - loss: 1.0290 - precision: 0.6647 - recall: 0.6647 - val_accuracy: 0.9464 - val_f1_score: 0.9204 - val_loss: 0.2687 - val_precision: 0.9464 - val_recall: 0.9464
Epoch 2/10
674/674 - 104s - 154ms/step - accuracy: 0.7115 - f1_score: 0.6957 - loss: 0.8337 - precision: 0.7115 - recall: 0.7115 - val_accuracy: 0.3988 - val_f1_score: 0.5347 - val_loss: 5.1339 - val_precision: 0.3988 - val_recall: 0.3988
Epoch 3/10
674/674 - 103s - 153ms/step - accuracy: 0.7718 - f1_score: 0.7657 - loss: 0.5471 - precision: 0.7718 - recall: 0.7718 - val_accuracy: 0.8036 - val_f1_score: 0.8495 - val_loss: 6.8223 - val_precision: 0.8036 - val_recall: 0.8036
Epoch 4/10
674/674 - 102s - 152ms/step - accuracy: 0.8550 - f1_score: 0.8530 - loss: 0.4813 - precision: 0.8550 - recall: 0.8550 - val_accuracy: 0.9286 - val_f1_score: 0.9188 - val_loss: 8.4016 - val_precision: 0.9286 - val_recall: 0.9286
Epoch 5/10
674/674 - 104s - 154ms/step - acc

In [20]:
if Drift_fine_tune_flag:

    if 'history_drift' in locals():
        del history_drift
    
    match StoppingSelector:
        case 'val_loss':
            StopCallback_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= 3,
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= 3,
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )
    
    #set only last layer to be trainable
    for layer in Drift_model.layers[-30:]:
        layer.trainable = True
    
    Drift_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), #start with "normal" learning rate
        loss=loss_fun_drift,
        #loss=tf.keras.losses.SparseCategoricalCrossentropy,
        #metrics=["accuracy",'precision',]
        weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
    )
    
    
    history_drift = Drift_model.fit(
        drift_model_gen,
        validation_data = drift_model_val_gen,
        epochs=10,
        class_weight = class_weights_drift,
        callbacks=[StopCallback_drift],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/10
674/674 - 215s - 319ms/step - accuracy: 0.5035 - f1_score: 0.5022 - loss: 1.5049 - precision: 0.5035 - recall: 0.5035 - val_accuracy: 0.0536 - val_f1_score: 0.0054 - val_loss: 73020.5938 - val_precision: 0.0536 - val_recall: 0.0536
Epoch 2/10
674/674 - 184s - 273ms/step - accuracy: 0.4624 - f1_score: 0.4622 - loss: 0.9944 - precision: 0.4624 - recall: 0.4624 - val_accuracy: 0.9464 - val_f1_score: 0.9204 - val_loss: 35.0972 - val_precision: 0.9464 - val_recall: 0.9464
Epoch 3/10
674/674 - 183s - 272ms/step - accuracy: 0.5455 - f1_score: 0.5451 - loss: 0.9435 - precision: 0.5455 - recall: 0.5455 - val_accuracy: 0.8393 - val_f1_score: 0.8696 - val_loss: 0.5157 - val_precision: 0.8393 - val_recall: 0.8393
Epoch 4/10
674/674 - 184s - 273ms/step - accuracy: 0.5004 - f1_score: 0.5003 - loss: 0.8599 - precision: 0.5004 - recall: 0.5004 - val_accuracy: 0.9464 - val_f1_score: 0.9204 - val_loss: 7.7183 - val_precision: 0.9464 - val_recall: 0.9464
Epoch 5/10
674/674 - 183s - 272ms/step 

In [21]:
if Drift_fine_tune_flag:

    if 'history_drift' in locals():
        del history_drift
    
    match StoppingSelector:
        case 'val_loss':
            StopCallback_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience=7,
                restore_best_weights=True,
                verbose=2,
                start_from_epoch=1
            )
        case 'val_f1':
            StopCallback_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience=7,
                restore_best_weights=True,
                verbose=2,
                mode='max'
            )
    
    #set only last layer to be trainable
    for layer in Drift_model.layers[-100:]:
        layer.trainable = True
    
    Drift_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  #start with "normal" learning rate
        loss=loss_fun_drift,
        #loss=tf.keras.losses.SparseCategoricalCrossentropy,
        #metrics=["accuracy",'precision',]
        weighted_metrics=["accuracy", 'precision', 'recall', tf.keras.metrics.F1Score(average='weighted')]
    )
    
    history_drift = Drift_model.fit(
        drift_model_gen,
        validation_data=drift_model_val_gen,
        epochs=50,
        class_weight=class_weights_drift,
        callbacks=[StopCallback_drift],
        verbose=2  #2 is one line per epoch -
    )

Epoch 1/50
674/674 - 232s - 344ms/step - accuracy: 0.5755 - f1_score: 0.5741 - loss: 0.7016 - precision: 0.5755 - recall: 0.5755 - val_accuracy: 0.9405 - val_f1_score: 0.9174 - val_loss: 1.6783 - val_precision: 0.9405 - val_recall: 0.9405
Epoch 2/50
674/674 - 195s - 289ms/step - accuracy: 0.6761 - f1_score: 0.6758 - loss: 0.6193 - precision: 0.6761 - recall: 0.6761 - val_accuracy: 0.8988 - val_f1_score: 0.8960 - val_loss: 0.6450 - val_precision: 0.8988 - val_recall: 0.8988
Epoch 3/50
674/674 - 195s - 289ms/step - accuracy: 0.6120 - f1_score: 0.6116 - loss: 0.6224 - precision: 0.6120 - recall: 0.6120 - val_accuracy: 0.8750 - val_f1_score: 0.8876 - val_loss: 0.6482 - val_precision: 0.8750 - val_recall: 0.8750
Epoch 4/50
674/674 - 195s - 290ms/step - accuracy: 0.6010 - f1_score: 0.6010 - loss: 0.6863 - precision: 0.6010 - recall: 0.6010 - val_accuracy: 0.9464 - val_f1_score: 0.9204 - val_loss: 0.4198 - val_precision: 0.9464 - val_recall: 0.9464
Epoch 5/50
674/674 - 195s - 290ms/step - acc

In [22]:
if Drift_fine_tune_flag:

    #make one folder for each model to save metrics
    model_InceptionV3_drift_dir = os.path.join(hyperparam_dir,'Inception_Drift_MultiPhase_fine_tune_2')
    if not os.path.isdir(model_InceptionV3_drift_dir):
        os.makedirs(model_InceptionV3_drift_dir)
    
    # test accuracy on test data - test accuracy ALWAYS on full test set
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = Drift_model.evaluate(test_generator_drift_bal)
    
        #for classification report
        true_labels = test_generator_drift_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = Drift_model.predict(test_generator_drift_metrics_bal)
    
    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = Drift_model.evaluate(test_generator_drift)
    
        true_labels = test_generator_drift_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = Drift_model.predict(test_generator_drift_metrics)
    
    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)
    
    print(f"Full fine tune Model:  Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")
    
    print(classification_report(true_labels, predicted_labels,target_names = label_list_drift))
    
    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list_drift,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()
    
    #save to file
    metrics_baseline_savename = os.path.join(model_InceptionV3_drift_dir,'classification_report.csv')
    
    report_df.to_csv(metrics_baseline_savename)
    
    #save model as well for future use
    #save the model:
    Drift_model.save(os.path.join(model_InceptionV3_drift_dir,'model.keras'))

211/211 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - accuracy: 0.9242 - f1_score: 0.8923 - loss: 0.4619 - precision: 0.9242 - recall: 0.9242
211/211 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step
Full fine tune Model:  Test Accuracy: 0.924 | Test Loss: 0.462 | Test Precision: 0.924 | Test Recall: 0.924 | Test F1 Score: 0.892:
              precision    recall  f1-score   support

    0_Normal       0.93      0.99      0.96       196
     1_Drift       0.00      0.00      0.00        15

    accuracy                           0.92       211
   macro avg       0.46      0.50      0.48       211
weighted avg       0.86      0.92      0.89       211



In [23]:
# try a simple model

model_2_drift = tf.keras.Sequential([
    tf.keras.layers.Input((target_size[0], target_size[1], 3)),  #image are greyscale - so in total dim (width,height,1)
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax"),
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_2 = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_2 = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

model_2_drift.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
    loss=loss_fun_2,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

In [24]:
if Drift_simple_flag:

    if 'history_m2_drift' in locals():
        del history_m2_drift

    match StoppingSelector:
        case 'val_loss':
            StopCallback_m2_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience=7,
                restore_best_weights=True,
                verbose=2,
                start_from_epoch=1
            )
        case 'val_f1':
            StopCallback_m2_drift = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience=7,
                restore_best_weights=True,
                verbose=2,
                mode='max'
            )


    history_m2_drift = model_2_drift.fit(
        drift_model_gen,
        validation_data=drift_model_val_gen,
        epochs=50,
        class_weight=class_weights_drift,
        callbacks=[StopCallback_m2_drift],
        verbose=2  #2 is one line per epoch -
    )